# Time Series Project

En la primera etapa nos ocupamos de cargar los datos

# CARGANDO EL DATASET

In [1]:
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

from paquete_proyecto.iniciando.bases import importar_databases
from paquete_proyecto.herramientas.data_info import data_info

In [2]:
ventas, ventas_sin_duplicados = importar_databases()

La longitud de la serie Nº 1 es de: 127255
La longitud de la serie Nº 2 es de: 108794


In [106]:
127979 - 127255

724

In [3]:
display(ventas.sample(3))
data_info(ventas, "ventas")

,IdCliente,NombreCliente,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
Fecha,,,,,,,,,,,,
2017-04-07,C860016819,PROVEEDOR Y SERCARGA S A,2.0,LL-PLC-0049,X MULTI Z 275/70R22.5,101.0,LLANTA,2,2391529,Engativa,Ventas externas,22.0
2019-05-12,C19193436,OLMEDO JOSE,44.0,LUA-0002,ELF EVOLUTION PRO 20W50 GALON 5 LITROS,103.0,LUBRICANTES,1,120000,Engativa,Calle 80,31.0
2019-01-18,C1016035566,HERRERA PAOLA,14.0,LL-2R-0007,CITY GRIP 130/70-12,101.0,LLANTA,1,106723,Suba,Suba,32.0


info de ventas,columna,type,count,NaN,NaN_pct,unique,unique_pct
0,IdCliente,object,127255,0,0.0 %,24122,18.96 %
1,NombreCliente,object,127255,0,0.0 %,21840,17.16 %
2,Empleado,float64,127255,0,0.0 %,56,0.04 %
3,Referencia,object,127255,0,0.0 %,1854,1.46 %
4,Descripcion,object,127255,0,0.0 %,1814,1.43 %
5,CodigoFamilia,float64,127255,0,0.0 %,5,0.0 %
6,Familia,object,127255,0,0.0 %,5,0.0 %
7,Cantidad,object,127255,0,0.0 %,213,0.17 %
8,Ventas,object,127255,0,0.0 %,14279,11.22 %
9,Localidad,object,127255,0,0.0 %,6,0.0 %


<div>
    <h4>Evaluación de cada columna</h4>
    <ol>
        <li> <b>IdCliente</b>: object - código alfanumérico</li><br />
        <li> <b>NombreCliente</b>: object - string = Nombre, Apellido</li><br />
        <li> <b>>> Empleado</b>: float - cambiar a int</li><br />
        <li> <b>Referencia</b>: object: categoría ordinal</li><br />
        <li> <b>Descripcion</b>: object: categoria cualitativa </li><br />
        <li> <b>>> CodigoFamilia</b>: float64: Cambiar a int</li><br />
        <li> <b>Familia</b>: object: categoria cualitativa</li><br />
        <li> <b>>>Cantidad</b>: object: cambiar a int</li><br />
        <li> <b>>> Ventas</b>: int: cambiar a float</li><br />
        <li> <b>Localidad</b>: object: categoria bussines</li><br />
        <li> <b>>> Area</b>: float: cambiar a int</li><br />
    </ol>
</div>

In [4]:
from paquete_proyecto.iniciando.variables import ajustar_tipos

In [5]:
ventas, ventas_sin_duplicados = ajustar_tipos(ventas), ajustar_tipos(ventas_sin_duplicados)

# CREANDO LAS REPRESENTACIONES TEMPORALES
Para cada una de las variables, debería haber una representación temporal, en el que cada registro represente una día unicamente

In [25]:
# crear una pivot table. que en el indice tenga fechas, en las columnas tenga localidad, y que los valores sean la cantidad de clientes que tuvo.

pd.pivot_table(ventas, values="IdCliente", index="Fecha", columns="Localidad", aggfunc="count")

Localidad,Barrios unidos,Bogota,Engativa,Puente aranda,Suba,Usaquen
Fecha,,,,,,
2016-01-09,7.0,16.0,68.0,11.0,19.0,11.0
2016-01-10,6.0,19.0,37.0,4.0,22.0,8.0
2016-01-11,7.0,17.0,100.0,29.0,13.0,5.0
2016-01-12,5.0,14.0,107.0,5.0,17.0,12.0
2016-02-09,3.0,1.0,46.0,33.0,11.0,7.0
...,...,...,...,...,...,...
2020-10-02,5.0,17.0,79.0,11.0,13.0,NaN
2020-10-03,3.0,NaN,41.0,20.0,16.0,8.0
2020-11-01,2.0,2.0,49.0,12.0,14.0,9.0


In [29]:
# crear una pivot table como la anterior, que en los values tenga cantidad de empleados
pd.pivot_table(ventas, values="Empleado", index="Fecha", columns="Localidad", aggfunc="count")

print ('Al parecer por cada transacción hay un empleado, no hay 2 transacciones realizadas por el mismo empleado en 1 dia')

Al parecer por cada transacción hay un empleado, no hay 2 transacciones realizadas por el mismo empleado en 1 dia


In [30]:
ventas.Familia.unique()

array(['LLANTA', 'SERVICIOS', 'REENCAUCHE', 'LUBRICANTES', 'FILTROS'],
      dtype=object)

# COMPLETANDO EL DATASET

Primero tenemos que armar cada una de las series temporales

Para poder aplicar Cross-Validation a la serie temporal, es necesario completar con las fechas faltantes con algun valor default en sus variables. De esta forma podremos garantizar que nuestro folds representan el mismo timedelta.

In [113]:
start_date = ventas.index.min()
end_date = ventas.index.max()

date_range = pd.date_range(start=start_date, end=end_date)
date_range

## HAY QUE TOMAR UN CRITERIO DE UNIFICACION DEL DATASET VENTAS: MIRAR TEST_2
df2 = ventas.reindex(date_range).fillna(0.0).rename_axis('Fecha').reset_index()
df2


ValueError: cannot reindex on an axis with duplicate labels

# ARMANDO MUESTREOS TEMPORALES

Vamos a hacer cross validation con scikit learn sobre un DataFrame de prueba

In [33]:
muestra = pd.DataFrame([0.0 for i in range(len(date_range))], index=date_range, columns=['default'])
muestra

,default
2016-01-09,0.0
2016-01-10,0.0
2016-01-11,0.0
2016-01-12,0.0
2016-01-13,0.0
...,...
2020-11-28,0.0
2020-11-29,0.0
2020-11-30,0.0
2020-12-01,0.0


In [71]:
from sklearn.model_selection import TimeSeriesSplit

In [91]:
# Son 1790 / 30 = 59,66 días por fold
# k = 29
# splitter = TimeSeriesSplit(n_splits=k)
splitter = TimeSeriesSplit(n_splits=29, max_train_size=60, test_size=60)

In [92]:
train = []
test = [] #only purpouse is to extract last element
for train_index, test_index in splitter.split(muestra.index):
    concat_index, predict_index = muestra.index[train_index], muestra.index[test_index]
    train.append(muestra.loc[concat_index])
    test.append(muestra.loc[predict_index])

# train.append(test[-1])

In [93]:
test[0]
len(train[3])
test[-1]
train[0]
len(train)

29

In [94]:
print (train[0].index.min())
print (train[0].index.max())

2016-01-09 00:00:00
2016-02-27 00:00:00


In [95]:
print (train[1].index.min())
print (train[1].index.max())

2016-02-28 00:00:00
2016-04-27 00:00:00


In [96]:
print (train[2].index.min())
print (train[2].index.max())

2016-04-28 00:00:00
2016-06-26 00:00:00


In [97]:
print (train[3].index.min())
print (train[3].index.max())

2016-06-27 00:00:00
2016-08-25 00:00:00


In [98]:
print (train[4].index.min())
print (train[4].index.max())

2016-08-26 00:00:00
2016-10-24 00:00:00


In [99]:
print (train[5].index.min())
print (train[5].index.max())

2016-10-25 00:00:00
2016-12-23 00:00:00


In [102]:
print (train[-1].index.min())
print (train[-1].index.max())

2020-08-05 00:00:00
2020-10-03 00:00:00


In [101]:
print (test[-1].index.min())
print (test[-1].index.max())

2020-10-04 00:00:00
2020-12-02 00:00:00


In [103]:
train.append(test[-1])